In [ ]:
original = "Test_RVE"
file = open(original + ".mvt",'r')
data = file.readlines()
data = [x.strip("\t") for x in data]

import pyDOE2
import numpy as np

#Create factorial design
strains = pyDOE2.gsd([3,3,3],2)
strains = strains + 1
strains = np.insert(strains, 2, 4, axis=1)
strains = np.insert(strains, 4, 4, axis=1)
strains = np.insert(strains, 5, 4, axis=1)

#Time Step array
ts = [0.001, 0.005, 0.01, 0.05, 0.1]

for i in range(len(strains)):
    for j in range(len(ts)):
        f = open(original + "v_" + str(i) + str(j) + ".mvt", "w+")
        for k in range(len(data)):
            if k == 6:
                f.write(data[k].replace("0.03", str(ts[j]), 3))
            elif k == 16:
                f.write(data[k].replace("33", str(int(1/ts[j])), 1))
            else:
                f.write(data[k].replace("\t1\t1\t1\t1\t1\t1", "\t" + str(strains[i,0]) + "\t" + str(strains[i,1]) + \
                                        "\t" + str(strains[i,2]) + "\t" + str(strains[i,3]) + \
                                        "\t" + str(strains[i,4]) + "\t" + str(strains[i,5])))



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('res/Test_RVE.csv')
  
#df.iloc[5,5]


fig, ax = plt.subplots()
#ax.plot(df[' Strain_11'], df[' Stress_11'])
ax.plot(df[' Strain_22'], df[' Stress_22'])
ax.plot(df[' Strain_33'], df[' Stress_33'])
ax.plot(df[' Strain_12'], df[' Stress_12'])
ax.plot(df[' Strain_13'], df[' Stress_13'])
ax.plot(df[' Strain_23'], df[' Stress_23'])

ax.set(xlabel='time (s)', ylabel='voltage (mV)',
       title='About as simple as it gets, folks')
ax.grid()


fig, ax1 = plt.subplots()
#ax1.plot(df[' Time'], df[' C_1111'])
ax1.plot(df[' Time'], df[' C_1122'])
ax1.plot(df[' Time'], df[' C_1133'])
ax1.plot(df[' Time'], df[' C_2222'])
ax1.plot(df[' Time'], df[' C_2233'])
ax1.plot(df[' Time'], df[' C_3333'])
ax1.plot(df[' Time'], df[' C_1212'])
ax1.plot(df[' Time'], df[' C_1313'])
ax1.plot(df[' Time'], df[' C_2323'])

ax1.set(xlabel='time (s)', ylabel='voltage (mV)',
       title='About as simple as it gets, folks')
ax1.grid()



In [6]:

def diagnostic(file):
    import numpy as np
    import re
    f = open(file,'r')
    data = f.readlines()
    data = data[17:]


    #Solutions steps
    lookup = "Solution Step"
    steps = []
    for num, line in enumerate(data):
        if lookup in line:
            steps.append(num)
    comp = steps

    if len(steps) > 1:
        for i in range(len(steps)-1):
            steps[i] = steps[i+1] - steps[i]
        steps.pop()
        steps = np.array(steps) - np.full(len(steps), 1)
    else:
        steps = [len(data) - 3]

    
    #Initialize contents of interest
    time = 0
    memory = 0
    completed = 0
    plastic_err = 0
    max_it = 0
    failed_strain = []
    loss = []
    error = []

    #Time/memory of run
    t = re.findall(r'\d+', data[-2])
    m = re.findall(r'\d+', data[-1])
    if len(t) == 2:
        time = int(t[0])*60 + int(t[1])
        memory = int(m[0])
    else:
        time = int(t[0])
        memory = int(m[0])

    
    #Check for solution completion
    end = re.findall(r'\d+', data[comp[-1]])
    if end[0] == end[1]:
        completed = 1

    #Plasticity convergence error
    p = data[comp[-1]+1]
    if p == "\n":
        plastic_err = 1

    # Non-convergence stats
    if completed == 0 & plastic_err == 0:
        max_it = 1
        for line in data:
            if "Strain increment applied:" in line:
                failed_strain = re.findall("\d+\.\d+", line)
            elif "Loss in homogenized tangent constitutive tensor" in line:
                loss = re.findall("\d+\.\d+", line)
            elif "Warning: Convergence not achieved" in line:
                error = re.findall("\d+\.\d+", line)
            


    failed_strain = [float(i) for i in failed_strain]
    loss = [float(i) for i in loss]
    error = [float(i) for i in error]

    return [time, memory, completed, plastic_err, max_it, failed_strain, loss, error]


diag_list = []
for i in range(5):
    f = "Test_RVE_" + str(i) + ".log"
    diag_list.append(diagnostic(f))



In [7]:
diag_list

[[141, 196684, 0, 0, 1, [], [], []],
 [141, 196684, 0, 0, 1, [], [], []],
 [141, 196684, 0, 0, 1, [], [], []],
 [141, 196684, 0, 0, 1, [], [], []],
 [141, 196684, 0, 0, 1, [], [], []]]